In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
jsonl_path = "data/training_datasets/verified_nomods.jsonl"
#jsonl_path = "data/training_datasets/verified_nocomments.jsonl"

df = pd.read_json(jsonl_path, lines=True)

In [3]:
df.head()

,filename,filetype,content
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([..."
1,cryptol/examples/AE.cry,cry,// WORK IN PROGRESS\n\n/*\nImplementation of t...
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...
4,cryptol/examples/builtin_lifting.cry,cry,"//builtins lift over tuples, seqs, and records..."


In [9]:
from preprocessing.interpreter_process import verify_df_row_with_cryptol
MOUNT_DIR = "/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files"
SERVER_URL = "http://localhost:8080"
print(verify_df_row_with_cryptol(df, 35, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL))
rows = []
for i in range(len(df)):
    repl_result = verify_df_row_with_cryptol(
            df, i, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL)
    rows.append({"filename" : df.iloc[i]["filename"], **repl_result["load_info"]})



{'host_path': '/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files/aes-mct-ecb_0kn9psvx.cry', 'container_relpath': 'files/aes-mct-ecb_0kn9psvx.cry', 'load_info': {'load_ok': True, 'file': 'files/aes-mct-ecb_0kn9psvx.cry', 'error': 'None', 'file_deps': ['Cryptol', 'SuiteB']}}


In [10]:
repl_results_df = pd.DataFrame(rows)
repl_results_df.head()

,filename,load_ok,file,error,file_deps
0,cryptol/examples/splitAt.cry,True,files/splitAt_63lzktqd.cry,None,[Cryptol]
1,cryptol/examples/AE.cry,True,files/AE_vytd5jc4.cry,None,"[AE__parameter, Cryptol]"
2,cryptol/examples/xor_cipher.cry,True,files/xor_cipher_tn40lndi.cry,None,[Cryptol]
3,cryptol/examples/zero_weird.cry,True,files/zero_weird_pzbhu55k.cry,None,[Cryptol]
4,cryptol/examples/builtin_lifting.cry,True,files/builtin_lifting_pti17lgj.cry,None,[Cryptol]


In [17]:
repl_results_df.to_json(f"data/training_datasets/syntax_check/nomods.jsonl", orient="records", lines=True)